致命错误：运用歌曲类别评分这个特征，其实在协同过滤中并没有什么效果，因为其实把一个歌单里的每一首歌的评分都设置为相同的，和都设置为1没有什么区别，余弦相似度相当于没考虑类别评分，所以这个特征在协同过滤中根本没什么用...  
发现了这个错误之后，我又看了一些论文，也有人已经做了LDA和协同过滤算法的融合，但是是将LDA的核心思想结合到协同过滤，而并非单纯地使用文本提取的特征，因此如果想要真的融合LDA进入协同过滤，还是得对这两个算法都有比较深刻的了解。  
(但是我明天就要大作业展示了...）因此这里就就把这个歌曲类别评分特征用在最后的推荐上。当找到了最接近的前k个歌单之后，返回的推荐歌曲列表会进行过滤，只下推荐歌曲评分和原歌单评分相同的歌曲（LDA分在相同的类别），以此达到更好的推荐。  
未来考虑再复现那篇论文的算法吧。  

In [ ]:
import pickle as pkl
import numpy as np
from gensim import corpora, models, similarities


def getIdDic(in_pkl):
    # 输入：pkl文件 key:value 输出：out_dic字典 内部id:原id
    in_dic = pkl.load(open(in_pkl, "rb"))
    out_dic = {}
    id = 0
    for (k, v) in in_dic.items():
        out_dic[id] = k
        id = id+1
    return out_dic


def getTopK(pid, sim_matrix, k, playlist_pkl):
    # 输入：歌单内部id 相似度矩阵 取相似的歌单数k 歌单字典 歌单原id:歌单名
    # 输出：打印前k个最相似的歌单原id、歌单名和相似度 并返回最相似的歌单
    playlist = pkl.load(open(playlist_pkl, "rb"))
    id2pid_dic = getIdDic(playlist_pkl)
    result_dic = {}
    for i in range(0, pid):
        sim_matrix[pid][i] = sim_matrix[i][pid]  # 补齐矩阵
    result_list = sim_matrix[pid].tolist()
    for i in range(0, len(result_list)):
        result_dic[id2pid_dic[i]] = result_list[i]  # result_dic 歌单原id:相似度
    sorted_result = []
    sorted_sim_value = []
    for i in sorted(result_dic, key=result_dic.__getitem__, reverse=True):
        sorted_result.append(i)
        sorted_sim_value.append(result_dic[i])
    print("与", id2pid_dic[pid], ":", playlist[id2pid_dic[pid]], " 最相似的歌单为:")
    for i in range(1, k+1):
        print(sorted_result[i], ":", playlist[sorted_result[i]],
              " 相似度为:", sorted_sim_value[i])  # 歌曲id：歌曲名 相似度
    return sorted_result[1:k+1]  # 除了本身的最接近k个的歌单


def getRecommendSong(topk, pid, playlist_pkl, song_pkl, listsong_pkl):
    # 打印最相似歌单中未出现在原歌单的歌曲id 歌曲名 创作者
    song = pkl.load(open(song_pkl, "rb"))
    id2pid_dic = getIdDic(playlist_pkl)
    list2song_dic = pkl.load(open(listsong_pkl, "rb"))
    song_dic = list2song_dic[id2pid_dic[pid]]
    list_category = int(float(list(song_dic.values())[0]))
    print("该歌单的类别为：", list_category, "  ", category2topic[list_category-1])
    print("推荐以下歌曲：")
    for i in topk:
        for (k, v) in list2song_dic[i].items():
            try:
                # 如果这首歌没有出现在原歌单 并且类别评分和原歌单相同
                if (k not in list2song_dic[id2pid_dic[pid]])and (int(float(v)) == list_category):
                    print(k, ":", song[k])
            except Exception as e:
                print("")


lda = models.ldamodel.LdaModel.load('./model/playlist_lda.model')  # 加载模型
dic = pkl.load(open("dic_splited_sentence.pkl", "rb"))  # 读入字典
dictionary = pkl.load(open("./model/dictionary.pkl", "rb"))
category2topic = []
for topic in lda.print_topics(num_topics=30, num_words=5):
    category2topic.append(topic[1])

sim_matrix = np.load("sim_matrix.npy")
for i in range(4, 5):
    topk = getTopK(i, sim_matrix, 10, "./playlist.pkl")
    getRecommendSong(topk, i, "./playlist.pkl", "./song.pkl", "./listsong.pkl")
    print("---------------------------------------------------")


结果：
![result.png](./result.png)